In [4]:
import os
import csv
import numpy as np
from scipy import ndimage as ndi
from skimage import io, img_as_float
from skimage.filters import gabor_kernel
from PIL import Image
from scipy.fftpack import dct as scipy_dct

# Définir le chemin vers le dossier Datasets
base_dir = os.path.expanduser("~/Desktop/gabor-filter/Datasets")


# Définir les noms des sous-dossiers
sub_dirs = ["NORMAL", "PNEUMONIA"]

# Créer les filtres Gabor
kernels = []
for theta in range(4):
    theta = theta / 4.0 * np.pi
    for sigma in (1, 3):
        for frequency in (0.05, 0.25):
            kernel = np.real(gabor_kernel(frequency, theta=theta, sigma_x=sigma, sigma_y=sigma))
            kernels.append(kernel)

# Fonction pour calculer les caractéristiques d'une image
def compute_feats(image, kernels):
    feats = np.zeros((len(kernels), 2), dtype=np.double)
    for k, kernel in enumerate(kernels):
        filtered = ndi.convolve(image, kernel, mode='wrap')
        feats[k, 0] = filtered.mean()
        feats[k, 1] = filtered.var()
    return feats

def compute_dct(image, num_coeffs=10):
    # Appliquer la DCT 2D sur l'image
    dct_matrix = scipy_dct(scipy_dct(image, axis=0, norm='ortho'), axis=1, norm='ortho')
    # Aplatir la matrice DCT en un vecteur 1D
    dct_flat = dct_matrix.flatten()
    important_coeffs = dct_flat[:num_coeffs]
    return important_coeffs

def compute_ft(image, num_coeffs=10):
    fft = np.fft.fft2(image)  # Calcul de la FFT 2D
    fft_magnitude = np.abs(fft)  # Magnitude de la FFT
    fft_flat = fft_magnitude.flatten()
    important_coeffs_ft = np.sort(fft_flat)[-num_coeffs:]  # Garder les `num_coeffs` plus grandes valeurs
    return important_coeffs_ft

# Chemin du fichier CSV
csv_file_path = os.path.expanduser("~/Desktop/gabor-filter/Datasets_features1.csv")

# Créer le fichier CSV et ajouter les en-têtes
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Numéro d'image", "Caractéristique[Gabor][DCT][Fourier][PHOG]", "Étiquette"])

    # Parcourir chaque sous-dossier pour traiter chaque image
    for sub_dir in sub_dirs:
        folder_path = os.path.join(base_dir, sub_dir)
        
        if os.path.exists(folder_path):
            # Parcourir toutes les images dans le dossier
            for filename in os.listdir(folder_path):
                if filename.endswith(('.png', '.jpg', '.jpeg')):
                    # Charger l'image en niveaux de gris
                    image_path = os.path.join(folder_path, filename)
                    image = img_as_float(io.imread(image_path, as_gray=True))
                    
                    # Calculer le vecteur de caractéristiques
                    feats = compute_feats(image, kernels)
                    dct = compute_dct(image, num_coeffs=20)
                    ft = compute_ft(image, num_coeffs=20)
                    
                    # Extraire le numéro de l'image et le label
                    print(f"Image file '{filename}' was filtered successfully")
                    numero_image = filename.split('_')[-1].split('.')[0]  # ex: "48" de "PNEUMONIA_48.jpg"
                    label_str = filename.split('_')[0]  # ex: "PNEUMONIA"
                    label = 0 if label_str == "NORMAL" else 1 # 0==nromal, 1==pneumonia
                    
                    # Convertir les caractéristiques en chaîne de caractères
                    feats_str = str(feats.tolist())
                    dct_str = str(dct.tolist())
                    ft_str = str(ft.tolist())

                    # Écrire la ligne dans le fichier CSV
                    writer.writerow([numero_image, [feats_str, dct_str, ft_str], label])

print(f"Les caractéristiques des images ont été enregistrées dans '{csv_file_path}'.")

Contenu de /Users/chawkibhd/Desktop/gabor-filter/Datasets/NORMAL: ['NORMAL_22.jpg', 'NORMAL_21.jpg', 'NORMAL_8.jpg', 'NORMAL_9.jpg', 'NORMAL_20.jpg', 'NORMAL_18.jpg', 'NORMAL_19.jpg', 'NORMAL_17.jpg', 'NORMAL_2.jpg', 'NORMAL_3.jpg', 'NORMAL_16.jpg', 'NORMAL_14.jpg', 'NORMAL_1.jpg', 'NORMAL_0.jpg', 'NORMAL_15.jpg', 'NORMAL_11.jpg', 'NORMAL_4.jpg', 'NORMAL_5.jpg', 'NORMAL_10.jpg', 'NORMAL_12.jpg', 'NORMAL_7.jpg', 'NORMAL_6.jpg', 'NORMAL_13.jpg']
Image file 'NORMAL_22.jpg' was filtered successfully
Contenu de /Users/chawkibhd/Desktop/gabor-filter/Datasets/NORMAL: ['NORMAL_22.jpg', 'NORMAL_21.jpg', 'NORMAL_8.jpg', 'NORMAL_9.jpg', 'NORMAL_20.jpg', 'NORMAL_18.jpg', 'NORMAL_19.jpg', 'NORMAL_17.jpg', 'NORMAL_2.jpg', 'NORMAL_3.jpg', 'NORMAL_16.jpg', 'NORMAL_14.jpg', 'NORMAL_1.jpg', 'NORMAL_0.jpg', 'NORMAL_15.jpg', 'NORMAL_11.jpg', 'NORMAL_4.jpg', 'NORMAL_5.jpg', 'NORMAL_10.jpg', 'NORMAL_12.jpg', 'NORMAL_7.jpg', 'NORMAL_6.jpg', 'NORMAL_13.jpg']
Image file 'NORMAL_21.jpg' was filtered successfull